# Chapter 14
# Working with spatial data
구글에서는 주로 **Visualising Geospatial data with Python**라고 표현한다

책에는 데이터에 대해서 다음과 같이 언급한다.

- **Spatial data** are often stored in special data structures (i.e., **not just data.frames**). The most commonly used format for spatial data is called a shapefile. Another common format is KML. There are many other formats, and while mastering the details of any of these formats is not realistic in this treatment, there are some important basic notions that one must have in order to work with spatial data.<br>

**즉, 익숙했던 DataFrame 형태가 아닌 것이다.**
- 추가로 책에서 나왔던 dataset인 CholeraDeaths 데이터에 대해서, R에서는 아래와 같은 형태로 데이터를 DataFrame으로 만든다.<br>
`CholeraDeaths@data`


### 교재 속 내용은 무시하고, 우리는 파이썬에서 유명한 **folium**이라는 라이브러리를 사용할 것이다

In [1]:
import os
import warnings
import folium
import pandas as pd

#### 간단하게 좌표값을 통해 지도를 그릴 수 있다.

In [2]:
m = folium.Map(location=[45.5236, -122.6750])
m

In [3]:
# currnet file directory
wd = os.getcwd()

dataset = "CholeraDeaths.csv"

#### 교재 속 데이터를 시각화해본다. (좌표값의 중앙값을 찾아서 그려본다.)

In [4]:
# read the dataset
CholeraDeaths=pd.read_csv(os.path.join(wd, dataset))
CholeraDeaths["coords.x1"] = CholeraDeaths["coords.x1"] / 10000
CholeraDeaths["coords.x2"] = CholeraDeaths["coords.x2"] / 10000

In [5]:
CholeraDeaths.head()

,Id,Count,coords.x1,coords.x2,optional
0,0,3,52.930874,18.103135,True
1,0,2,52.931216,18.102517,True
2,0,1,52.931438,18.102029,True
3,0,1,52.931738,18.101426,True
4,0,4,52.932068,18.100787,True


In [6]:
CholeraDeaths.describe()

,Id,Count,coords.x1,coords.x2
count,250.0,250.000000,250.000000,250.000000
mean,0.0,1.956000,52.941385,18.103379
std,0.0,1.573521,0.010828,0.008454
min,0.0,1.000000,52.916034,18.085795
25%,0.0,1.000000,52.933217,18.097185
50%,0.0,1.000000,52.942253,18.102713
75%,0.0,2.000000,52.948879,18.109373
max,0.0,15.000000,52.965589,18.130621


In [7]:
x1 = CholeraDeaths["coords.x1"].median()
x1

52.9422525931865

In [8]:
x2 = CholeraDeaths["coords.x2"].median()
x2

18.102712929012

#### 하지만, 책 속에 등장한 지역의 좌표는 대략 (51.513333, -0.136667)이므로 아예 다른 지역 coordinate이다.
또 무언가 이상하다.

In [9]:
default_location=[x1, x2]
default_zoom_start=13
m = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
m

### 하지만 그냥 위 지도에다가 반복문을 통해 원을 mark 한다.
지도에 크기에 따른 마킹을 하는 방법에서는 이것이 최선인 것 같다.

In [10]:
for index, row in CholeraDeaths.iterrows():
    folium.CircleMarker(
    location=[row["coords.x1"], row["coords.x2"]],
    radius=row["Count"],
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
    ).add_to(m)

In [11]:
m

### folium에서 제공하는 기능들은 다음과 같다.
- Marker
- Circle
- CircleMarker
- LatLngPopup
- ClickForMarker
- Vincent/Vega and Altair/VegaLite Markers
- GeoJSON and TopoJSON layers
- Choropleth maps

[링크](https://python-visualization.github.io/folium/quickstart.html#Getting-Started)
#### 위 링크를 눌러서 들어가보면 다양한 기능을 확인할 수 있다.


## Marker

In [12]:
folium.Marker(
    location=[52.94, 18.10],
    popup='Mt. Hood Meadows',
    icon=folium.Icon(icon='cloud')
).add_to(m)

folium.Marker(
    location=[52.95, 18.10],
    popup='Timberline Lodge',
    icon=folium.Icon(color='green')
).add_to(m)

folium.Marker(
    location=[52.96, 18.10],
    popup='Some Other Location',
    icon=folium.Icon(color='red', icon='info-sign')
).add_to(m)

In [13]:
m

## Choropleth maps
folium 예시에서 나온 데이터를 사용하여 지도를 그려본다.

In [14]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
state_geo = f'{url}/us-states.json'
state_unemployment = f'{url}/US_Unemployment_Oct2012.csv'
state_data = pd.read_csv(state_unemployment)

In [15]:
state_data.head()

,State,Unemployment
0,AL,7.1
1,AK,6.8
2,AZ,8.1
3,AR,7.2
4,CA,10.1


#### State의 정보에 대해서는 state_geo URL에 들어가보면 json 형태의 데이터를 확인할 수 있다.
{"type":"FeatureCollection","features":[
{"type":"Feature","id":"AL","properties":{"name":"Alabama"},"geometry":{"type":"Polygon","coordinates":[[[-87.359296,35.00118],[-85.606675,34.984749],[-85.431413,34.124869],[-85.184951,32.859696],[-85.069935,32.580372],[-84.960397,32.421541],[-85.004212,32.322956],[-84.889196,32.262709],[-85.058981,32.13674],[-85.053504,32.01077],[-85.141136,31.840985],[-85.042551,31.539753],[-85.113751,31.27686],[-85.004212,31.003013],[-85.497137,30.997536],[-87.600282,30.997536],[-87.633143,30.86609],[-87.408589,30.674397],[-87.446927,30.510088],[-87.37025,30.427934],[-87.518128,30.280057],[-87.655051,30.247195],[-87.90699,30.411504],[-87.934375,30.657966],[-88.011052,30.685351],[-88.10416,30.499135],[-88.137022,30.318396],[-88.394438,30.367688],[-88.471115,31.895754],[-88.241084,33.796253],[-88.098683,34.891641],[-88.202745,34.995703],[-87.359296,35.00118]]]}}, ...

In [16]:
m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=state_data,
    columns=['State', 'Unemployment'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)'
).add_to(m)

folium.LayerControl().add_to(m)
m.add_child(folium.LatLngPopup())

m

### 콜로라도 주의 json 데이터 내의 좌표값을 살펴봐서 찍어보면 다음과 같다.
즉, 데이터가 제대로 오버레이할만큼 잘 정리되어 있어야 할 수 있는 것이다.

In [17]:
Colorado = [[-107.919731,41.003906],[-105.728954,40.998429],[-104.053011,41.003906],[-102.053927,41.003906],
            [-102.053927,40.001626],[-102.042974,36.994786],[-103.001438,37.000263],[-104.337812,36.994786],
            [-106.868158,36.994786],[-107.421329,37.000263],[-109.042503,37.000263],[-109.042503,38.166851],
            [-109.058934,38.27639],[-109.053457,39.125316],[-109.04798,40.998429],[-107.919731,41.003906]]


for i, j in Colorado:
    folium.CircleMarker(
    location=[j, i],
    radius=1,
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
    ).add_to(m)

m

### 또한, 교재에 나온 다른 그래프를 그리기 위해서는
### 또 다른 기능을 제공하는 라이브러리를 찾아야 한다.

#### 추가로  Heatmap

In [19]:
from folium.plugins import HeatMap

x1 = CholeraDeaths["coords.x1"].median()
x2 = CholeraDeaths["coords.x2"].median()
print(x1, x2)
m = folium.Map(location=[x1, x2], control_scale=True, zoom_start=12)

data = CholeraDeaths[['coords.x1', 
                      'coords.x2', 
                      'Count']].groupby(['coords.x1', 
                                         'coords.x2']).sum().reset_index().values.tolist()

HeatMap(data=data, radius=8, max_zoom=13).add_to(m)

m

52.9422525931865 18.102712929012
